In [1]:
from kafka import KafkaAdminClient, KafkaProducer, KafkaConsumer

# admin

In [2]:
broker = "localhost:9092"
admin = KafkaAdminClient(bootstrap_servers=[broker])

In [3]:
admin.list_topics()

[]

In [4]:
from kafka.admin import NewTopic

In [11]:
# admin.delete_topics(["even_nums"])

In [14]:
from kafka.errors import TopicAlreadyExistsError

In [15]:
try:
    admin.create_topics([NewTopic(name="even_nums", num_partitions=1, replication_factor=1)])
except TopicAlreadyExistsError:
    print("already exists")

already exists


In [16]:
try:
    admin.create_topics([NewTopic(name="odd_nums", num_partitions=2, replication_factor=1)])
except TopicAlreadyExistsError:
    print("already exists")

In [17]:
admin.list_topics()

['even_nums', 'odd_nums']

# producer

In [18]:
producer = KafkaProducer(bootstrap_servers=[broker])

In [21]:
result = producer.send("even_nums", bytes(str(0), "utf-8"))

In [22]:
result

In [23]:
result.get()

RecordMetadata(topic='even_nums', partition=0, topic_partition=TopicPartition(topic='even_nums', partition=0), offset=0, timestamp=1700078043741, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=1, serialized_header_size=-1)

In [24]:
from threading import Thread, Lock

lock = Lock()
def Print(*args):
    with lock:
        print(*args)

Print("hi")

hi


In [28]:
import time, threading

def num_producer(topic, start, step):
    producer = KafkaProducer(bootstrap_servers=[broker])
    num = start
    while True:
        if num < 10:
            Print("send", num, "to", topic)
        producer.send(topic, bytes(str(num), "utf-8"))
        num += step
        time.sleep(1)

threading.Thread(target=num_producer, args=("even_nums", 0, 2)).start()
threading.Thread(target=num_producer, args=("odd_nums", 1, 2)).start()

send 0 to even_nums
send 1 to odd_nums
send 2 to even_nums
send 3 to odd_nums
send 4 to even_nums
send 5 to odd_nums
send 6 to even_nums
send 7 to odd_nums
send 8 to even_nums
send 9 to odd_nums


# consumer

In [29]:
consumer = KafkaConsumer(bootstrap_servers=[broker])

In [30]:
batch = consumer.poll(1000)
batch

{}

In [31]:
consumer.assignment()

set()

## Manual Partition Assignment

In [33]:
from kafka import TopicPartition

In [47]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
consumer.assign([TopicPartition("even_nums", 0)])
consumer.seek_to_beginning()
consumer.assignment()

{TopicPartition(topic='even_nums', partition=0)}

In [50]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

674
676


## Automatic Partition Assignment

In [51]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
consumer.subscribe(["even_nums"])
print(consumer.assignment())

set()


In [54]:
_ = consumer.poll(1000)
print(consumer.assignment())
consumer.seek_to_beginning()

{TopicPartition(topic='even_nums', partition=0)}


In [57]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

914


# Multiple Assignment

In [58]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
consumer.subscribe(["even_nums", "odd_nums"])
print(consumer.assignment())

_ = consumer.poll(1000)
print(consumer.assignment())
consumer.seek_to_beginning()

set()
{TopicPartition(topic='odd_nums', partition=0), TopicPartition(topic='odd_nums', partition=1), TopicPartition(topic='even_nums', partition=0)}


In [64]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    print(topic_partition)
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

TopicPartition(topic='even_nums', partition=0)
1188
1190
1192
1194
1196
TopicPartition(topic='odd_nums', partition=1)
1189
1193
1195
1197
TopicPartition(topic='odd_nums', partition=0)
1191
